WARNING
=========
<hr />

## Guys, excuse me for my poor English. it's a foreign language for me, and I will try hard to write in it as good as it is possible.

### If you want to see the implementation details, please refer to [my github repo](https://github.com/stolpa4/Coursera_Capstone/tree/master/Week3Assignment) with all the code.

<hr />

Step 1: Notebook created!!
--------------------------------------

<hr />

Step 2: Scrape the [wiki page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) and create a DataFrame
-----------------------------------------------------------------------------------------------------------------------------------

#### Import some useful stuff:

In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup # Great web scraping library!

#### Download and parse the web-page:

In [2]:
page_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_response = requests.get(page_link, timeout=5) # Use GET to obtain the response
page_content = BeautifulSoup(page_response.content, 'html.parser') # Use html parser to parse the page

#### Let's extract the table from the content and build a pandas DataFrame from it:

To make it easy for you I put all the overwhelming code to module `wiki_table_extractor`. You can check it out in the [project's github repo](https://github.com/stolpa4/Coursera_Capstone/tree/master/Week3Assignment).

For this task we will use a specially prepared class - `WikiTableExtractor`. Let's import it.

In [3]:
from wiki_table_extractor import WikiTableExtractor

This class' method `parse_table_from_page()` takes one argument - our `BeautifulSoup` instance and parses it to `Table` object (described below). Now we will proceed to instantiate the class and use the described method.

In [4]:
table_extractor = WikiTableExtractor()
table_extractor.parse_table_from_page(page_content)
extracted_table = table_extractor.table

Our `Table` object - `extracted_table` consists of two lists, one with table titles and another with table rows. We can use a method of this object - `as_dict_list()` to obtain a list of dictionaries with table rows.
Let's obtain the list and use it to create a pandas DataFrame object.

In [5]:
table_dict_list = extracted_table.as_dict_list()
canada_post_codes = pd.DataFrame(data=table_dict_list)
print('DataFrame shape: ', canada_post_codes.shape)
canada_post_codes.head(10)

DataFrame shape:  (289, 3)


,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
8,Queen's Park,Not assigned,M7A
9,Not assigned,Not assigned,M8A


#### Now it's time to refine our DataFrame!

At first, let's get rid of all these 'Not assigned' postal codes!  
Again, here the code is encapsulated in the module named `utils`. If you are interested in implementation details, please, check out [this module](https://github.com/stolpa4/Coursera_Capstone/blob/master/Week3Assignment/utils.py).

In [6]:
import utils

In [7]:
canada_post_codes = utils.filter_not_assigned_postcodes(canada_post_codes)
print('DataFrame shape: ', canada_post_codes.shape)
canada_post_codes.head(10)

DataFrame shape:  (212, 3)


,Borough,Neighbourhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A
5,North York,Lawrence Manor,M6A
6,Queen's Park,Not assigned,M7A
7,Etobicoke,Islington Avenue,M9A
8,Scarborough,Rouge,M1B
9,Scarborough,Malvern,M1B


The next step is to name all neighborhoods that are 'Not assigned' (in fact, there is only one) after the corresponding Boroughs.  
To notice the result, pay attention to the row 6.

In [8]:
canada_post_codes = utils.name_not_assigned_neighborhoods(canada_post_codes)
canada_post_codes.head(10)

,Borough,Neighbourhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A
5,North York,Lawrence Manor,M6A
6,Queen's Park,Queen's Park,M7A
7,Etobicoke,Islington Avenue,M9A
8,Scarborough,Rouge,M1B
9,Scarborough,Malvern,M1B


Finally we need to combine the rows with the same post-code into one row with the neighborhoods separated with a comma.

In [9]:
canada_post_codes = utils.combine_rows_with_same_postcode(canada_post_codes)
print('DataFrame shape: ', canada_post_codes.shape)
canada_post_codes.tail(10)

DataFrame shape:  (103, 3)


,Postcode,Borough,Neighbourhood
93,M9A,Etobicoke,Islington Avenue
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery, Humberlea"
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


Now our data is structured properly and we can proceed to further steps.

<hr />

Step 4: Get the latitude and the longitude coordinates of each neighborhood 
--------------------------------------------------------------------------------------------------------------

#### let's use geocoder implicitly (code is in the util library, but in fact it's just a copy of the code from the assignment description)

In [10]:
latitude_column = []
longitude_column = []

for index, row in canada_post_codes.iterrows():
    latitude, longitude = utils.get_latitude_longitude(row['Postcode'])
    latitude_column.append(latitude)
    longitude_column.append(longitude)

Now, after we obtained lists with latitudes and longitudes of the neighborhoods, let's include them to our dataframe.

In [11]:
canada_post_codes['Latitude'] = latitude_column
canada_post_codes['Longitude'] = longitude_column
print('DataFrame shape: ', canada_post_codes.shape)
canada_post_codes.head(10)

DataFrame shape:  (103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",0,0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",0,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0,0
3,M1G,Scarborough,Woburn,0,0
4,M1H,Scarborough,Cedarbrae,0,0
5,M1J,Scarborough,Scarborough Village,0,0
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",0,0
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",0,0
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",0,0
9,M1N,Scarborough,"Birch Cliff, Cliffside West",0,0


In [12]:
WRONG_LATITUDES = canada_post_codes['Latitude'] == 0
WRONG_LONGITUDES = canada_post_codes['Longitude'] == 0

print('Wrong latitudes count: ', len(canada_post_codes[WRONG_LATITUDES]))
print('Wrong longitudes count: ', len(canada_post_codes[WRONG_LONGITUDES]))

Wrong latitudes count:  103
Wrong longitudes count:  103


Oh no! There must be some problem with geocoder library (or maybe I misused it, plesase check the code and let me know). Let's try again using CSV file with the coordinates

In [13]:
latitude_column = []
longitude_column = []

for index, row in canada_post_codes.iterrows():
    latitude, longitude = utils.get_latitude_longitude_from_csv(row['Postcode'])
    latitude_column.append(latitude)
    longitude_column.append(longitude)

In [14]:
canada_post_codes['Latitude'] = latitude_column
canada_post_codes['Longitude'] = longitude_column
print('DataFrame shape: ', canada_post_codes.shape)
canada_post_codes.head(10)

DataFrame shape:  (103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761
5,M1J,Scarborough,Scarborough Village,43.7447342,-79.2394761
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279292,-79.2620294
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111117,-79.2845772
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.2394761
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.2648481


Let's check that there is no zero coordinates:

In [15]:
WRONG_LATITUDES = canada_post_codes['Latitude'] == 0
WRONG_LONGITUDES = canada_post_codes['Longitude'] == 0

print('Wrong latitudes count: ', len(canada_post_codes[WRONG_LATITUDES]))
print('Wrong longitudes count: ', len(canada_post_codes[WRONG_LONGITUDES]))

Wrong latitudes count:  0
Wrong longitudes count:  0


<hr />

Final step: Exploring and clustering neighborhoods 
--------------------------------------------------------------------------

#### Let's work with only boroughs that contain the word Toronto.

In [16]:
WITH_TORONTO_WORD = canada_post_codes['Borough'].str.contains('Toronto')
canada_post_codes = canada_post_codes[WITH_TORONTO_WORD]
print('DataFrame shape: ', canada_post_codes.shape)
canada_post_codes.head(10)

DataFrame shape:  (38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.6763574,-79.2930312
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6795571,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.6689985,-79.3155716
43,M4M,East Toronto,Studio District,43.6595255,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.7280205,-79.3887901
45,M4P,Central Toronto,Davisville North,43.7127511,-79.3901975
46,M4R,Central Toronto,North Toronto West,43.7153834,-79.4056784
47,M4S,Central Toronto,Davisville,43.7043244,-79.3887901
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.6895743,-79.3831599
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.6864123,-79.4000493


#### Explore Neighborhoods in Torronto

Let's replicate (and refactor a little bit) the `getNearbyVenues()`  function from the lab. I'm going to define it in the module `utils`. If you are interested in the code - [check GitHub repo](https://github.com/stolpa4/Coursera_Capstone/tree/master/Week3Assignment). Now we create a data frame `toronto_venues`.

In [17]:
toronto_venues = pd.DataFrame()
for index, row in canada_post_codes.iterrows():
    venues_list = utils.get_nearby_venues(row['Neighbourhood'],
                                          row['Latitude'],
                                          row['Longitude'])
    toronto_venues = toronto_venues.append(pd.DataFrame(venues_list))

print('DataFrame shape: ', toronto_venues.shape)
toronto_venues.head(10)

DataFrame shape:  (1696, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
0,The Beaches,43.6763574,-79.2930312,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,The Beaches,43.6763574,-79.2930312,Grover Pub and Grub,Pub,43.679181,-79.297215
2,The Beaches,43.6763574,-79.2930312,Starbucks,Coffee Shop,43.678798,-79.298045
3,The Beaches,43.6763574,-79.2930312,Upper Beaches,Neighborhood,43.680563,-79.292869
0,"The Danforth West, Riverdale",43.6795571,-79.352188,Pantheon,Greek Restaurant,43.677621,-79.351434
1,"The Danforth West, Riverdale",43.6795571,-79.352188,Dolce Gelato,Ice Cream Shop,43.677773,-79.351187
2,"The Danforth West, Riverdale",43.6795571,-79.352188,MenEssentials,Cosmetics Shop,43.677820,-79.351265
3,"The Danforth West, Riverdale",43.6795571,-79.352188,Messini Authentic Gyros,Greek Restaurant,43.677827,-79.350569
4,"The Danforth West, Riverdale",43.6795571,-79.352188,Cafe Fiorentina,Italian Restaurant,43.677743,-79.350115
5,"The Danforth West, Riverdale",43.6795571,-79.352188,Mezes,Greek Restaurant,43.677962,-79.350196


Let's check how many venues were returned for each neighbourhood

In [18]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",20,20,20,20,20,20
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",13,13,13,13,13,13
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,81,81,81,81,81,81
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


How many unique categories can be curated from all the returned venues?

In [19]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 unique categories.


#### Analyze each neighbourhood

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

Print each neighbourhood along with 5 most common venues

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Italian Restaurant  0.04
3              Bakery  0.04
4                 Pub  0.04


----Brockton, Exhibition Place, Parkdale Village----
               venue  freq
0     Breakfast Spot  0.10
1        Coffee Shop  0.10
2               Café  0.10
3                Bar  0.05
4  Convenience Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0         Yoga Studio  0.06
1      Farmers Market  0.06
2  Light Rail Station  0.06
3       Garden Center  0.06
4              Garden  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0      Airport Lounge  0.15
1     Air

Let's put this to pandas DataFrame

In [23]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = utils.get_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Asian Restaurant,Restaurant,Hotel,Burger Joint,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Café,Pub,Cheese Shop,Seafood Restaurant,Restaurant,Bakery,Italian Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Pet Store,Convenience Store,Nightclub,Climbing Gym,Caribbean Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Recording Studio
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boat or Ferry


Finally, let's cluster neighborhoods!

In [24]:
import sklearn.cluster as ct

# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

toronto_grouped_clustering

# run k-means clustering
kmeans = ct.KMeans(n_clusters=kclusters, random_state=125).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([7, 7, 0, 7, 8, 7, 7, 0, 9, 7, 7, 7, 7, 7, 7, 0, 7, 6, 0, 7, 7, 0,
       4, 0, 1, 7, 7, 2, 3, 7, 7, 7, 7, 7, 7, 5, 7, 7], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = canada_post_codes

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.6763574,-79.2930312,5,Health Food Store,Neighborhood,Coffee Shop,Pub,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6795571,-79.352188,7,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.6689985,-79.3155716,7,Park,Sandwich Place,Italian Restaurant,Pet Store,Gym,Pub,Movie Theater,Burrito Place,Burger Joint,Brewery
43,M4M,East Toronto,Studio District,43.6595255,-79.340923,7,Café,Coffee Shop,Gastropub,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Park,Seafood Restaurant,Sandwich Place
44,M4N,Central Toronto,Lawrence Park,43.7280205,-79.3887901,4,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


Let's get coordinates of Toronto

In [26]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='toronto_explorer')

location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


Let's create a map with the resulting clusters

In [27]:
import folium
from matplotlib import cm, colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7
    ).add_to(map_clusters)
    
map_clusters.save('map_clusters.html')

And visualize it!

In [28]:
from IPython.display import HTML, display

with open('map_clusters.html') as f_map:
    m = f_map.read()
map_height = 600
srcdoc = m.replace('"', '&quot;')
embed = HTML('<iframe srcdoc="{}" '
             'style="height: {}px; display:block; width: 100%; margin: 0 auto; '
             'border: none"></iframe>'.format(srcdoc, map_height))
embed

#### Examine Clusters

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Downtown Toronto,0,Café,Bar,Japanese Restaurant,Bookstore,Restaurant,Bakery,Coffee Shop,Pub,Beer Bar,Beer Store
67,Downtown Toronto,0,Café,Bar,Vegetarian / Vegan Restaurant,Chinese Restaurant,Vietnamese Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Caribbean Restaurant,Dessert Shop
76,West Toronto,0,Bakery,Pharmacy,Discount Store,Supermarket,Gym / Fitness Center,Pool,Music Venue,Middle Eastern Restaurant,Café,Brewery
77,West Toronto,0,Bar,Coffee Shop,Asian Restaurant,Restaurant,Vietnamese Restaurant,Café,Bakery,Pizza Place,Cocktail Bar,Men's Store
78,West Toronto,0,Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Pet Store,Convenience Store,Nightclub,Climbing Gym,Caribbean Restaurant
82,West Toronto,0,Mexican Restaurant,Café,Bookstore,Furniture / Home Store,Music Venue,Diner,Cajun / Creole Restaurant,Sandwich Place,Fast Food Restaurant,Flea Market


In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Playground,Summer Camp,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,2,Park,Playground,Trail,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Home Service,Garden,Music Venue,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,5,Health Food Store,Neighborhood,Coffee Shop,Pub,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Central Toronto,6,Mexican Restaurant,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,7,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
42,East Toronto,7,Park,Sandwich Place,Italian Restaurant,Pet Store,Gym,Pub,Movie Theater,Burrito Place,Burger Joint,Brewery
43,East Toronto,7,Café,Coffee Shop,Gastropub,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Park,Seafood Restaurant,Sandwich Place
45,Central Toronto,7,Breakfast Spot,Hotel,Clothing Store,Food & Drink Shop,Dance Studio,Sandwich Place,Burger Joint,Park,Gym,Gym / Fitness Center
46,Central Toronto,7,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Italian Restaurant,Gym / Fitness Center,Chinese Restaurant,Dessert Shop,Rental Car Location,Diner
47,Central Toronto,7,Dessert Shop,Sandwich Place,Pizza Place,Restaurant,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Deli / Bodega,Diner
49,Central Toronto,7,Pub,Coffee Shop,Pizza Place,Spa,Bagel Shop,Fried Chicken Joint,Sports Bar,Supermarket,American Restaurant,Sushi Restaurant
51,Downtown Toronto,7,Coffee Shop,Restaurant,Park,Café,Italian Restaurant,Bakery,Pub,Pizza Place,Pharmacy,Breakfast Spot
52,Downtown Toronto,7,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Café,Men's Store,Fast Food Restaurant,Gastropub,Burger Joint
53,Downtown Toronto,7,Coffee Shop,Park,Bakery,Café,Pub,Theater,Breakfast Spot,Mexican Restaurant,Brewery,Shoe Store


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Downtown Toronto,8,Airport Lounge,Airport Service,Airport Terminal,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boat or Ferry


In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Downtown Toronto,9,Grocery Store,Café,Park,Italian Restaurant,Convenience Store,Baby Store,Diner,Restaurant,Coffee Shop,Nightclub
